Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE".

In [1]:
# Please add your name and University of Michagan uniqname here...

NAME = 'Paul Natland'
UMICH_UNIQNAME = 'pnatland'

In [2]:
"""
ACKNOWLEDGEMENTS: For this assignment, I benefited from lecture videos, searches on Stacks Overflow & RapidTables, and conversations on SLACK
"""

'\nACKNOWLEDGEMENTS: For this assignment, I benefited from lecture videos, searches on Stacks Overflow & RapidTables, and conversations on SLACK\n'

---

In [3]:
version = "v2.3.060120"

# SIADS 515 Week 3 Homework (HW3)


Think of this assignment in the following way:  it's your first day on the job and you've been given a notebook that was authored by someone who is no longer with your company.  You've been asked to fix it.  There are errors in it, and some of it was not completed by the original author.  You're lucky, though, as there are assertions
sprinkled throughout the notebook to help guide you along the way.


Top-level goal of notebook:
Read a CSV file into a pandas DataFrame and add specific columns to it.
These columns are added by applying functions to specific columns.
The columns to add include:
1. A datetime column that converts "Garmin time" to standard (unix epoch) time.  Note that Garmin doesn't use standard epoch offsets for their timestamps.  Rather than using the number of seconds that have elapsed since midnight on January 1, 1970, they use the number of seconds from midnight on December 31, 1989.

2. A conversion of "semicircles" of latitude and longitude to two different formats: degrees, minutes, seconds 3-tuples and fractional degrees.  For example, a latitude of 504719750 semicircles corresponds to a 3-tuple of degrees, minutes and seconds of (42, 18, 18.43) and 42.305121 degrees.

3. A "normalized speed" column that consists of the values for speed modified to remove outliers by replacing them with upper and lower bounds as well as normalized to z-values (i.e. by subtracting the mean from each value and dividing the result by the standard deviation).

In addition, you will need to complete a function that looks at the difference between sequential rows to determine whether the cyclist is slowing down or not.

Your task for this assignment is to debug this notebook to produce the desired results as shown in the assertions below.



In [4]:
#corrected import statement
import pandas as pd

In [5]:
#corrected import statement
import numpy as np

In [6]:
ride = pd.read_csv('assets/ride_final2.csv')

In [7]:
ride.head()

,Timestamp,Latitude,Longitude,Distance,Altitude,Speed
0,896018545,504719750,-998493490,10.87,285.8,1.773
1,896018560,504717676,-998501870,71.85,285.0,5.533
2,896018566,504716354,-998506792,108.02,284.0,6.485
3,896018575,504714055,-998515244,170.23,284.0,6.951
4,896018584,504711900,-998523278,229.27,285.0,6.224


# Answer to #1

In [8]:
def garmin_time_to_datetime(series):
    """Convert Garmin FIT time by adding the number of 
    seconds from January 1, 1970 to December 31, 1989.
    """
    #corrected the value for the number of seconds from 64000000 s
    return pd.to_datetime(series + 631065600, unit='s')

In [9]:
ride['Timestamp_datetime'] = ride.Timestamp.map(garmin_time_to_datetime)

In [10]:
ride.head()

,Timestamp,Latitude,Longitude,Distance,Altitude,Speed,Timestamp_datetime
0,896018545,504719750,-998493490,10.87,285.8,1.773,2018-05-23 14:02:25
1,896018560,504717676,-998501870,71.85,285.0,5.533,2018-05-23 14:02:40
2,896018566,504716354,-998506792,108.02,284.0,6.485,2018-05-23 14:02:46
3,896018575,504714055,-998515244,170.23,284.0,6.951,2018-05-23 14:02:55
4,896018584,504711900,-998523278,229.27,285.0,6.224,2018-05-23 14:03:04


# Answer to #2

In [11]:
from numpy import datetime64
assert ride.Timestamp_datetime.values[0] == datetime64('2018-05-23T14:02:25.000000000')

In [12]:
# solution
def semicircles_to_degrees(semicircles):
    '''
    Convert semicircles to degrees
    '''
    #below changed from 2**32 based on conversions found online
    max_31_bit_int = 2**31
    return semicircles * (180/max_31_bit_int)

In [13]:
#original code provided
# def semicircles_to_degrees(semicircles):
#     '''
#     Convert semicircles to degrees
#     '''
#     max_32_bit_int = 2**32
#     return semicircles * (180/max_32_bit_int)

In [14]:
#solution
def degrees_to_dms(degrees_fraction):
    ''' Convert degrees to degree, minute, second 3-tuples'''
    degrees = int(degrees_fraction)
    #should be int from research online (e.g. RapidTables)
    minutes = int((degrees_fraction - degrees) * 60)
    seconds = np.round((degrees_fraction - degrees - minutes/60) * 3600,2)
    return (degrees, abs(minutes), abs(seconds))

In [15]:
#original code provided
# def degrees_to_dms(degrees_fraction):
#     ''' Convert degrees to degree, minute, second 3-tuples'''
#     degrees = int(degrees_fraction)
#     minutes = (degrees_fraction - degrees) * 60
#     seconds = np.round((degrees_fraction - degrees - minutes/60) * 3600,2)
#     return (degrees, abs(minutes), abs(seconds))

In [16]:
def dms_to_degrees(d,m,s):
    ''' Convert degrees, minutes, seconds to fractional degrees'''
    return d+m/60+s/3600

In [17]:
dms_to_degrees(*degrees_to_dms(42.2833333333))

42.28333333333333

In [18]:
assert dms_to_degrees(*degrees_to_dms(42.2833333333)) == 42.28333333333333

In [19]:
ride['Latitude_degrees'] = ride['Latitude'].map(semicircles_to_degrees)

In [20]:
ride['Longitude_degrees'] = ride['Longitude'].map(semicircles_to_degrees)

In [21]:
ride['Latitude_dms'] = ride['Latitude_degrees'].map(degrees_to_dms)

In [22]:
ride['Longitude_dms'] = ride['Longitude_degrees'].map(degrees_to_dms)

In [23]:
ride.head()

,Timestamp,Latitude,Longitude,Distance,Altitude,Speed,Timestamp_datetime,Latitude_degrees,Longitude_degrees,Latitude_dms,Longitude_dms
0,896018545,504719750,-998493490,10.87,285.8,1.773,2018-05-23 14:02:25,42.305121,-83.692758,"(42, 18, 18.43)","(-83, 41, 33.93)"
1,896018560,504717676,-998501870,71.85,285.0,5.533,2018-05-23 14:02:40,42.304947,-83.693460,"(42, 18, 17.81)","(-83, 41, 36.46)"
2,896018566,504716354,-998506792,108.02,284.0,6.485,2018-05-23 14:02:46,42.304836,-83.693872,"(42, 18, 17.41)","(-83, 41, 37.94)"
3,896018575,504714055,-998515244,170.23,284.0,6.951,2018-05-23 14:02:55,42.304643,-83.694581,"(42, 18, 16.72)","(-83, 41, 40.49)"
4,896018584,504711900,-998523278,229.27,285.0,6.224,2018-05-23 14:03:04,42.304463,-83.695254,"(42, 18, 16.07)","(-83, 41, 42.92)"


In [24]:
ride.tail()

,Timestamp,Latitude,Longitude,Distance,Altitude,Speed,Timestamp_datetime,Latitude_degrees,Longitude_degrees,Latitude_dms,Longitude_dms
209,896019779,504424374,-999047484,5930.37,278.2,4.740,2018-05-23 14:22:59,42.280363,-83.739193,"(42, 16, 49.31)","(-83, 44, 21.09)"
210,896019785,504424487,-999051230,5955.96,278.0,4.059,2018-05-23 14:23:05,42.280372,-83.739507,"(42, 16, 49.34)","(-83, 44, 22.22)"
211,896019788,504425416,-999051046,5965.33,278.0,3.042,2018-05-23 14:23:08,42.280450,-83.739491,"(42, 16, 49.62)","(-83, 44, 22.17)"
212,896019791,504425984,-999050165,5973.35,278.0,2.267,2018-05-23 14:23:11,42.280498,-83.739417,"(42, 16, 49.79)","(-83, 44, 21.9)"
213,896019805,504426837,-999050457,5981.77,278.2,0.000,2018-05-23 14:23:25,42.280569,-83.739442,"(42, 16, 50.05)","(-83, 44, 21.99)"


# Answer to #3

In [25]:
from numpy import round
last_row = ride.iloc[213]
assert round(last_row.Latitude_degrees,6) == 42.280569
assert round(last_row.Longitude_degrees,6) == -83.739442

assert last_row.Latitude_dms == (42, 16, 50.05)
assert last_row.Longitude_dms == (-83,44,21.99)


## this was the original assert before the correction noted on SLACK ##
#assert last_row.Latitude_dms == (42,17,9.95)

In [26]:
# original code provided
# def normalize(df, pd_series_name, nsd=2)
#     '''
#     Take all values that are outside some bound (mean +- 2 sd by default)
#     and convert them to the appropriate bound.
#     Scale the results to z-scores before returning them
#     '''
#     df = df.copy()
#     pd_series = df[pd_series_name].astype(float)

#     # Find upper and lower bound for outliers
#     avg = np.mean(pd_series)
#     sd  = np.std(pd_series)

#     # Calculate the bounds
#     lower_bound = avg - nsd*sd
#     upper_bound = avg + nsd*sd

#     # Collapse in the outliers: replace them with appropriate bound
#     df.loc[pd_series < lower_bound , pd_series_name ] = upper_bound
#     df.loc[pd_series > upper_bound , pd_series_name ] = upper_bound
    
#     return (df[pd_series_name] - avg) / sd

In [27]:
# solution

##missing colon in definition initialization
def normalize(df, pd_series_name, nsd=2):
    '''
    Take all values that are outside some bound (mean +- 2 sd by default)
    and convert them to the appropriate bound.
    Scale the results to z-scores before returning them
    '''
    ## wanted to make copy of different name if using .copy() method
    df_copy = df.copy()
    pd_series = df_copy[pd_series_name].astype(float)

    # Find upper and lower bound for outliers
    avg = np.mean(pd_series)
    sd  = np.std(pd_series)

    # Calculate the bounds
    lower_bound = avg - nsd*sd
    upper_bound = avg + nsd*sd

    # Collapse in the outliers: replace them with appropriate bound
    ##the top statement said upper_bound but was corrected to lower_bound
    df_copy.loc[pd_series < lower_bound , pd_series_name ] = lower_bound
    df_copy.loc[pd_series > upper_bound , pd_series_name ] = upper_bound
    
    return (df_copy[pd_series_name] - avg) / sd

In [28]:
ride['Speed_normalized'] = normalize(ride,'Speed')

In [29]:
#looking into data created
display(ride['Speed_normalized'].max(), ride['Speed_normalized'].min(), ride['Speed_normalized'].value_counts())

2.0

-2.0

 2.000000    7
-2.000000    3
 0.989570    3
-0.134973    3
 0.511595    3
            ..
 1.924335    1
 0.462165    1
 0.750803    1
-1.147855    1
 1.368242    1
Name: Speed_normalized, Length: 176, dtype: int64

# Created Function

In [30]:
from numpy import round
assert round(ride.iloc[0].Speed_normalized,4) == -1.7737
assert ride.iloc[213].Speed_normalized == -2.0

In [31]:
# What was initially provided
# def proportion_slowing(df,series_name):
#     ''' Calculate the proportion of rows that represent a slower speed than the previous row'''
#     return 0 # whoops -- ran out of time to do this before I got fired!

### This is my first solution

In [32]:
def proportion_slowing(df,series_name):
    ''' Calculate the proportion of rows that represent a slower speed than the previous row'''
    count = 0
    initial_index = df.index[0]
    for i in range(len(df[series_name])):
        if i==0:
            pass
        elif df[series_name][i+initial_index] < df[series_name][(i-1)+initial_index]:
            count +=1
    return count/len(df[series_name])

In [33]:
%timeit proportion_slowing(ride,'Speed_normalized')

4 ms ± 48.1 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


### This is my second solution which was much faster -- the .diff() method made a big difference

In [34]:
def proportion_slowing(df,series_name):
    ''' Calculate the proportion of rows that represent a slower speed than the previous row'''
    return len(df.index[df[series_name].diff() < 0])/len(df[series_name])

In [35]:
%timeit proportion_slowing(ride,'Speed_normalized')

321 µs ± 2.6 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [36]:
from numpy import round
assert round(proportion_slowing(ride,'Speed_normalized'),6) == 0.514019
assert round(proportion_slowing(ride[:10],'Speed_normalized'),6) == 0.4
assert round(proportion_slowing(ride[10:],'Speed_normalized'),6) == 0.519608

In [37]:
## these are the original assertions before the correction indicated on SLACK ##
# from numpy import round
# assert round(proportion_slowing(ride,'Speed_normalized'),6) == 0.514019
# assert round(proportion_slowing(ride[:10],'Speed_normalized'),6) == 0.018692
# assert round(proportion_slowing(ride[10:],'Speed_normalized'),6) == 0.495327

In [38]:
# This cell is here to assign points for your submission. The content will be manually graded.
assert True == True, "Assignment 3 has been submitted."

### This is an attempting to improve the %timeit for the result -- about 2 orders of magnitude faster than my faster function above

In [39]:
speeds_normalized_tuple = tuple(ride.Speed_normalized.values)

In [40]:
from functools import lru_cache

@lru_cache(16)
def proportion_slowing2(tuple):
    ''' Calculate the proportion of rows that represent a slower speed than the previous row'''
    diff_list = [x - tuple[i - 1] for i, x in enumerate(tuple) if i > 0]
    count = 0
    for value in diff_list:
        if value < 0:
            count += 1
    return count/len(tuple)

In [41]:
%timeit proportion_slowing2(speeds_normalized_tuple)

4.01 µs ± 38.8 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [42]:
%timeit proportion_slowing2.__wrapped__(speeds_normalized_tuple)

79.9 µs ± 463 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
